In [1]:
# import libraries.
import torch
import numpy as np
import datetime
# tf.enable_eager_execution()
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiplicativeLR, StepLR, LambdaLR
import h5py
import glob
import matplotlib.pyplot as plt
import time
import os
import torch.optim as optim
from natsort import natsorted
print('imported')
# #######################



imported


# Caution! change foldername before execution

In [2]:
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = 'D:/hale'
# Data_path = '/content/drive/My Drive/'
# Data_path = os.getcwd()[0:-5]
ckpt_folder = '1'

checkpoint_path = os.path.normpath(os.path.join(Data_path,'results','checkpoints',ckpt_folder))
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
files = glob.glob(os.path.normpath(os.path.join(checkpoint_path,'*')))
for f in files:
    os.remove(f)
    
loss_path = os.path.normpath(os.path.join(Data_path,'results','losses',ckpt_folder))
if not os.path.exists(loss_path):
    os.makedirs(loss_path)
files = glob.glob(os.path.normpath(os.path.join(loss_path,'*')))
for f in files:
    os.remove(f)
    
time_path = os.path.normpath(os.path.join(Data_path,'results','times',ckpt_folder))
if not os.path.exists(time_path):
    os.makedirs(time_path)
files = glob.glob(os.path.normpath(os.path.join(time_path,'*')))
for f in files:
    os.remove(f)
    
timestep = 16
batch_size = 32
n_features = 257
epochs_num = 100
w=7


In [0]:
files

['/home/speech/h_damirchi/du16/results/checkpoints/322/bests']

In [6]:
class Dataset_loader():

    def __init__(self, path, batch_size, timestep, X_, Y_):
        self.write_path = path
        self.batch_size = batch_size
        self.len_data = 0
        self.X_ = X_
        self.Y_ = Y_
        self.timestep = timestep
        # with h5py.File(os.path.normpath(os.path.join(self.write_path))+'.hdf5', "r") as f:
        #     obj = f['X']
        #     self.len_data = obj.shape[0]
        #     self.n_features = obj.shape[1]
        self.len_data = X_.shape[0]
        self.n_features = Y_.shape[1]
        self.index_arr = np.arange(self.len_data)
        self.index_arr = self.index_arr[0:(len(self.index_arr)//self.timestep)*self.timestep]
        self.index_arr = self.index_arr.reshape((len(self.index_arr)//self.timestep,self.timestep))
        np.random.shuffle(self.index_arr)
        
    def get_len(self):
        return len(self.X_)

    def check_if_left(self):
        if len(self.index_arr)>=self.batch_size:
            return True
        return False

    def get_batch(self):
        self.X_out=self.X_[self.index_arr[0:self.batch_size].reshape(1,self.batch_size*self.timestep)]
        self.Y_out=self.Y_[self.index_arr[0:self.batch_size].reshape(1,self.batch_size*self.timestep)]
        self.index_arr = self.index_arr[self.batch_size:]
#         print(self.X_out.reshape(self.batch_size,self.timestep,self.n_features))
#         print(self.X_out.reshape(self.batch_size,self.timestep,self.n_features))
        return self.X_out.reshape(self.batch_size,self.timestep,self.n_features),self.Y_out.reshape(self.batch_size,self.timestep,self.n_features)

In [3]:
path = os.path.normpath(os.path.join(Data_path,'h5_files_e','30hdata_2','features','data_30h_loge'))
path_val = os.path.normpath(os.path.join(Data_path,'h5_files_e','devdata_3','features','devdata_30h_loge'))

with h5py.File(path+'.hdf5', "r") as f:
    X = f['X'][:]
with h5py.File(path+'.hdf5', "r") as f:
    Y = f['Y'][:]
with h5py.File(path_val+'.hdf5', "r") as f:
    X_val = f['X'][:]
with h5py.File(path_val+'.hdf5', "r") as f:
    Y_val = f['Y'][:]

In [7]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=n_features,hidden_size=257, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(514, 257)
    def forward(self, x):
        x, _  = self.lstm(x)
        x = self.fc(x)
        return x
    
net = Net()
net.cuda()
net.train()
###################################
def loss_func(y_true, y_pred, cov):
    a = (y_true - y_pred).reshape(batch_size,16,1,257)
    b = torch.inverse(cov).reshape(1,1,257,257).repeat(batch_size,16,1,1)
    c = (y_true - y_pred).reshape(batch_size,16,257,1)
    a = a.reshape((batch_size*16,1,257))
    b = b.reshape((batch_size*16,257,257))
    c = c.reshape((batch_size*16,257,1))
    loss = torch.sum(torch.einsum('mpq,mqf->mpf',torch.einsum('mpq,mqz->mpz', a, b),c))   
    loss = loss.reshape((batch_size,16,1,1))
    return loss
def loss_func_val(y_true, y_pred):
    loss = torch.mean(torch.pow(y_true-y_pred,2))                                                                  
    return loss
###############################
optimizer = optim.Adam(net.parameters(), lr=0.0001)
############################
#################################
n_features = 257
train_loss = np.zeros((epochs_num,1))
val_loss = np.zeros((epochs_num,1))+np.inf
start = time.time()
load_batch = Dataset_loader(path, batch_size, timestep, X, Y)
cov = torch.tensor(np.identity(n_features), device = torch.device("cuda:0"), dtype=torch.float32, requires_grad=False)
id_tens = torch.tensor(np.identity(n_features), device = torch.device("cuda:0"), dtype=torch.float32)
# criterion = custom_losses()
criterion = loss_func
criterion_val = loss_func_val
len_data = load_batch.get_len()
num_data = (((load_batch.get_len()//16)*16)//batch_size)*batch_size
for epoch in range(epochs_num): 
    running_loss = 0.0
    loss_tot = 0.0
    v_loss_tot = 0.0
    i=0
    load_batch = Dataset_loader(path, batch_size, timestep, X, Y)
    load_batch_cov = Dataset_loader(path, batch_size, timestep, X, Y)
    load_batch_val = Dataset_loader(path_val, batch_size, timestep, X_val, Y_val)
    net.train()
    while load_batch.check_if_left():
        inputs, labels = load_batch.get_batch()
        inputs=Variable(torch.tensor(inputs).float())
        labels=Variable(torch.tensor(labels).float())
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels,cov)
        
        loss.backward(retain_graph=True) 
        optimizer.step()

        running_loss += loss.item()
        loss_tot += loss.item()
        if (i+1) % 100 == 0:   
            print('[epoch %d, minibatch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        i = i+1
    cov = torch.tensor(np.zeros((257,257)), device = torch.device("cuda:0"), dtype=torch.float32, requires_grad=False)
    x=0
    while load_batch_cov.check_if_left():
        net.eval()
        with torch.no_grad():
            inputs, labels = load_batch_cov.get_batch()
            inputs=Variable(torch.tensor(inputs, dtype=torch.float32, requires_grad=False))
            labels=Variable(torch.tensor(labels, dtype=torch.float32, requires_grad=False))
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = net(inputs)
            aa = (labels - outputs).reshape(batch_size,16,1,257).transpose(2,3)
            bb = (labels - outputs).reshape(batch_size,16,1,257)
            aa = aa.reshape((batch_size*16,257,1))
            bb = bb.reshape((batch_size*16,1,257))
            x = torch.sum(torch.sum(torch.einsum('mpq,mqf->mpf', aa, bb),axis=0),axis=0)
            x = x.reshape(batch_size,16,1)
            cov = cov.add(x)
    
    cov = cov.diag().diag()/num_data
    
    train_loss[epoch] = loss_tot/i
    print('epoch %d, train loss: %.3f' %(epoch+1, loss_tot/i))
    net.eval()
    i=0
    while load_batch_val.check_if_left():
        inputs_val, labels_val = load_batch_val.get_batch()
        inputs_val=Variable(torch.tensor(inputs_val).float())
        labels_val=Variable(torch.tensor(labels_val).float())
        inputs_val = inputs_val.cuda()
        labels_val = labels_val.cuda()
        outputs_val = net(inputs_val)
        v_loss= criterion_val(outputs_val, labels_val)
        v_loss_tot += v_loss.item()
        i=i+1
    if (v_loss_tot/i)<np.min(val_loss):
        torch.save(net.state_dict(), os.path.normpath(os.path.join(checkpoint_path,'ckpt_'+str(epoch+1)+'.pth')))
    val_loss[epoch] = v_loss_tot/i
    print('************ epoch %d, val loss: %.3f ************' %(epoch+1, v_loss_tot/i))
print('Finished Training')
stop = time.time()
with open((os.path.normpath(os.path.join(time_path,str(ckpt_folder)+'.txt'))), 'w') as f:
    f.write('%d' % int(stop-start))
np.savetxt(os.path.normpath(os.path.join(loss_path,'train_loss.txt')),train_loss)
np.savetxt(os.path.normpath(os.path.join(loss_path,'val_loss.txt')),val_loss)
model_path = os.path.normpath(os.path.join(Data_path,'results','models',str(ckpt_folder)+'.pth'))
torch.save(net.state_dict(), model_path)
print('Finished saving')

[epoch 1, minibatch   100] loss: 9143610.998
[epoch 1, minibatch   200] loss: 3605341.155
[epoch 1, minibatch   300] loss: 2581890.210
[epoch 1, minibatch   400] loss: 1972401.164
[epoch 1, minibatch   500] loss: 1603814.394
[epoch 1, minibatch   600] loss: 1388611.052
[epoch 1, minibatch   700] loss: 1247437.186
[epoch 1, minibatch   800] loss: 1130714.123
[epoch 1, minibatch   900] loss: 1088995.386
[epoch 1, minibatch  1000] loss: 1059940.565
[epoch 1, minibatch  1100] loss: 978341.664
[epoch 1, minibatch  1200] loss: 999630.213
[epoch 1, minibatch  1300] loss: 957935.963
[epoch 1, minibatch  1400] loss: 927269.517
[epoch 1, minibatch  1500] loss: 901152.994
[epoch 1, minibatch  1600] loss: 921998.268
[epoch 1, minibatch  1700] loss: 886751.056
[epoch 1, minibatch  1800] loss: 871464.963
[epoch 1, minibatch  1900] loss: 857505.289
[epoch 1, minibatch  2000] loss: 815625.220
[epoch 1, minibatch  2100] loss: 833600.134
[epoch 1, minibatch  2200] loss: 813291.291
[epoch 1, minibatch  2

[epoch 2, minibatch  3700] loss: 124967.626
[epoch 2, minibatch  3800] loss: 128022.137
[epoch 2, minibatch  3900] loss: 127109.057
[epoch 2, minibatch  4000] loss: 126879.124
[epoch 2, minibatch  4100] loss: 124716.281
[epoch 2, minibatch  4200] loss: 124512.384
[epoch 2, minibatch  4300] loss: 126222.045
[epoch 2, minibatch  4400] loss: 125417.251
[epoch 2, minibatch  4500] loss: 126908.360
[epoch 2, minibatch  4600] loss: 125626.602
[epoch 2, minibatch  4700] loss: 129378.385
[epoch 2, minibatch  4800] loss: 125236.714
[epoch 2, minibatch  4900] loss: 124263.861
[epoch 2, minibatch  5000] loss: 124008.878
[epoch 2, minibatch  5100] loss: 133866.537
[epoch 2, minibatch  5200] loss: 119791.189
[epoch 2, minibatch  5300] loss: 123630.840
[epoch 2, minibatch  5400] loss: 125205.790
[epoch 2, minibatch  5500] loss: 126102.328
[epoch 2, minibatch  5600] loss: 123331.638
[epoch 2, minibatch  5700] loss: 123872.203
[epoch 2, minibatch  5800] loss: 121314.841
[epoch 2, minibatch  5900] loss:

[epoch 3, minibatch  7300] loss: 127528.070
[epoch 3, minibatch  7400] loss: 126186.663
[epoch 3, minibatch  7500] loss: 124645.736
[epoch 3, minibatch  7600] loss: 124887.144
[epoch 3, minibatch  7700] loss: 123832.212
[epoch 3, minibatch  7800] loss: 128177.742
[epoch 3, minibatch  7900] loss: 129707.610
[epoch 3, minibatch  8000] loss: 122650.839
[epoch 3, minibatch  8100] loss: 127058.806
[epoch 3, minibatch  8200] loss: 127144.251
[epoch 3, minibatch  8300] loss: 125565.636
[epoch 3, minibatch  8400] loss: 129876.379
[epoch 3, minibatch  8500] loss: 124255.883
[epoch 3, minibatch  8600] loss: 123722.117
[epoch 3, minibatch  8700] loss: 126611.029
[epoch 3, minibatch  8800] loss: 122743.831
[epoch 3, minibatch  8900] loss: 122519.638
[epoch 3, minibatch  9000] loss: 124427.354
[epoch 3, minibatch  9100] loss: 133377.042
[epoch 3, minibatch  9200] loss: 126225.976
[epoch 3, minibatch  9300] loss: 126719.868
[epoch 3, minibatch  9400] loss: 127279.085
[epoch 3, minibatch  9500] loss:

[epoch 4, minibatch 10900] loss: 127558.847
[epoch 4, minibatch 11000] loss: 126547.324
[epoch 4, minibatch 11100] loss: 134446.160
[epoch 4, minibatch 11200] loss: 129839.055
[epoch 4, minibatch 11300] loss: 130663.169
[epoch 4, minibatch 11400] loss: 122124.475
[epoch 4, minibatch 11500] loss: 126268.392
[epoch 4, minibatch 11600] loss: 124129.865
[epoch 4, minibatch 11700] loss: 126077.088
[epoch 4, minibatch 11800] loss: 125576.600
[epoch 4, minibatch 11900] loss: 123098.809
[epoch 4, minibatch 12000] loss: 126034.081
[epoch 4, minibatch 12100] loss: 125511.133
[epoch 4, minibatch 12200] loss: 126298.329
[epoch 4, minibatch 12300] loss: 127517.361
[epoch 4, minibatch 12400] loss: 125286.655
[epoch 4, minibatch 12500] loss: 125463.175
[epoch 4, minibatch 12600] loss: 123241.028
[epoch 4, minibatch 12700] loss: 130074.571
[epoch 4, minibatch 12800] loss: 126663.644
[epoch 4, minibatch 12900] loss: 131258.950
[epoch 4, minibatch 13000] loss: 126365.140
[epoch 4, minibatch 13100] loss:

[epoch 5, minibatch 14500] loss: 130743.902
[epoch 5, minibatch 14600] loss: 133893.799
[epoch 5, minibatch 14700] loss: 129287.684
[epoch 5, minibatch 14800] loss: 126907.732
[epoch 5, minibatch 14900] loss: 131678.889
epoch 5, train loss: 131205.059
************ epoch 5, val loss: 5.212 ************
[epoch 6, minibatch   100] loss: 132045.005
[epoch 6, minibatch   200] loss: 131565.884
[epoch 6, minibatch   300] loss: 132136.847
[epoch 6, minibatch   400] loss: 132054.751
[epoch 6, minibatch   500] loss: 130189.629
[epoch 6, minibatch   600] loss: 132894.217
[epoch 6, minibatch   700] loss: 130415.780
[epoch 6, minibatch   800] loss: 127532.337
[epoch 6, minibatch   900] loss: 135130.412
[epoch 6, minibatch  1000] loss: 132055.046
[epoch 6, minibatch  1100] loss: 129830.283
[epoch 6, minibatch  1200] loss: 130617.886
[epoch 6, minibatch  1300] loss: 133439.743
[epoch 6, minibatch  1400] loss: 130291.061
[epoch 6, minibatch  1500] loss: 132527.823
[epoch 6, minibatch  1600] loss: 1325

...

[epoch 95, minibatch 11000] loss: 133347.661
[epoch 95, minibatch 11100] loss: 131711.558
[epoch 95, minibatch 11200] loss: 131683.365
[epoch 95, minibatch 11300] loss: 132937.328
[epoch 95, minibatch 11400] loss: 130643.017
[epoch 95, minibatch 11500] loss: 133095.225
[epoch 95, minibatch 11600] loss: 133557.816
[epoch 95, minibatch 11700] loss: 135148.176
[epoch 95, minibatch 11800] loss: 130931.887
[epoch 95, minibatch 11900] loss: 134167.212
[epoch 95, minibatch 12000] loss: 131964.618
[epoch 95, minibatch 12100] loss: 129661.501
[epoch 95, minibatch 12200] loss: 132730.957
[epoch 95, minibatch 12300] loss: 133265.762
[epoch 95, minibatch 12400] loss: 133857.520
[epoch 95, minibatch 12500] loss: 133008.945
[epoch 95, minibatch 12600] loss: 130504.959
[epoch 95, minibatch 12700] loss: 130125.559
[epoch 95, minibatch 12800] loss: 132693.246
[epoch 95, minibatch 12900] loss: 133133.424
[epoch 95, minibatch 13000] loss: 133588.564
[epoch 95, minibatch 13100] loss: 131882.763
[epoch 95,

[epoch 96, minibatch 14200] loss: 131901.979
[epoch 96, minibatch 14300] loss: 134899.943
[epoch 96, minibatch 14400] loss: 134939.238
[epoch 96, minibatch 14500] loss: 133806.424
[epoch 96, minibatch 14600] loss: 134864.088
[epoch 96, minibatch 14700] loss: 133076.164
[epoch 96, minibatch 14800] loss: 131247.055
[epoch 96, minibatch 14900] loss: 129467.011
epoch 96, train loss: 132963.231
************ epoch 96, val loss: 4.507 ************
[epoch 97, minibatch   100] loss: 131587.466
[epoch 97, minibatch   200] loss: 130080.447
[epoch 97, minibatch   300] loss: 130310.773
[epoch 97, minibatch   400] loss: 129433.367
[epoch 97, minibatch   500] loss: 128776.997
[epoch 97, minibatch   600] loss: 130992.008
[epoch 97, minibatch   700] loss: 132205.130
[epoch 97, minibatch   800] loss: 130565.315
[epoch 97, minibatch   900] loss: 133426.343
[epoch 97, minibatch  1000] loss: 132146.032
[epoch 97, minibatch  1100] loss: 132811.489
[epoch 97, minibatch  1200] loss: 129711.656
[epoch 97, mini

[epoch 98, minibatch  2300] loss: 129369.742
[epoch 98, minibatch  2400] loss: 130360.742
[epoch 98, minibatch  2500] loss: 129667.075
[epoch 98, minibatch  2600] loss: 130718.721
[epoch 98, minibatch  2700] loss: 128557.371
[epoch 98, minibatch  2800] loss: 130400.293
[epoch 98, minibatch  2900] loss: 130163.236
[epoch 98, minibatch  3000] loss: 131497.295
[epoch 98, minibatch  3100] loss: 131019.566
[epoch 98, minibatch  3200] loss: 130245.027
[epoch 98, minibatch  3300] loss: 129268.598
[epoch 98, minibatch  3400] loss: 131641.664
[epoch 98, minibatch  3500] loss: 133057.313
[epoch 98, minibatch  3600] loss: 131243.420
[epoch 98, minibatch  3700] loss: 132868.290
[epoch 98, minibatch  3800] loss: 130680.517
[epoch 98, minibatch  3900] loss: 130780.790
[epoch 98, minibatch  4000] loss: 132167.625
[epoch 98, minibatch  4100] loss: 131504.417
[epoch 98, minibatch  4200] loss: 131846.762
[epoch 98, minibatch  4300] loss: 133117.312
[epoch 98, minibatch  4400] loss: 132272.064
[epoch 98,

[epoch 99, minibatch  5500] loss: 133322.680
[epoch 99, minibatch  5600] loss: 131441.300
[epoch 99, minibatch  5700] loss: 133000.291
[epoch 99, minibatch  5800] loss: 134654.585
[epoch 99, minibatch  5900] loss: 130799.380
[epoch 99, minibatch  6000] loss: 131555.330
[epoch 99, minibatch  6100] loss: 132764.507
[epoch 99, minibatch  6200] loss: 133409.493
[epoch 99, minibatch  6300] loss: 129719.645
[epoch 99, minibatch  6400] loss: 131610.017
[epoch 99, minibatch  6500] loss: 133661.278
[epoch 99, minibatch  6600] loss: 130862.445
[epoch 99, minibatch  6700] loss: 133472.542
[epoch 99, minibatch  6800] loss: 132707.619
[epoch 99, minibatch  6900] loss: 131467.693
[epoch 99, minibatch  7000] loss: 132300.535
[epoch 99, minibatch  7100] loss: 131884.776
[epoch 99, minibatch  7200] loss: 129400.300
[epoch 99, minibatch  7300] loss: 132399.420
[epoch 99, minibatch  7400] loss: 133018.369
[epoch 99, minibatch  7500] loss: 133532.249
[epoch 99, minibatch  7600] loss: 131397.326
[epoch 99,

[epoch 100, minibatch  8500] loss: 134102.027
[epoch 100, minibatch  8600] loss: 134404.043
[epoch 100, minibatch  8700] loss: 133494.350
[epoch 100, minibatch  8800] loss: 132554.839
[epoch 100, minibatch  8900] loss: 134133.152
[epoch 100, minibatch  9000] loss: 130825.086
[epoch 100, minibatch  9100] loss: 131841.625
[epoch 100, minibatch  9200] loss: 130058.788
[epoch 100, minibatch  9300] loss: 130993.256
[epoch 100, minibatch  9400] loss: 131455.762
[epoch 100, minibatch  9500] loss: 132591.269
[epoch 100, minibatch  9600] loss: 130119.277
[epoch 100, minibatch  9700] loss: 132704.291
[epoch 100, minibatch  9800] loss: 129916.867
[epoch 100, minibatch  9900] loss: 130637.349
[epoch 100, minibatch 10000] loss: 130762.696
[epoch 100, minibatch 10100] loss: 131783.658
[epoch 100, minibatch 10200] loss: 131622.197
[epoch 100, minibatch 10300] loss: 130434.040
[epoch 100, minibatch 10400] loss: 129925.910
[epoch 100, minibatch 10500] loss: 132863.478
[epoch 100, minibatch 10600] loss: